In [ ]:
import numpy as np
import igl
import meshplot as mp
import scipy.sparse as sp
import json

# Prepare Data

In [ ]:
v, f = igl.read_triangle_mesh('data/star.off')
v -= v.min(axis=0)
v /= v.max()

In [ ]:
plot = mp.plot(v, f, shading={"wireframe": True})

In [ ]:
with open("./data/star_meta.txt", "r") as file:
    meta = json.load(file)

cage_n = meta['cage_n']
star_n = meta['star_n']

cage_indices = np.array(meta['cage_indices'])
star_indices = np.array(meta['star_indices'])

cage_points = v[cage_indices]
star_points = v[star_indices]

sides = meta['cage_sides']

In [ ]:
# function to display cage + shape only (not the entire mesh)
def display(cage_n, cage_points, star_n, star_points):

    points = np.vstack([cage_points, star_points])
    
    cage_edges = [[i, (i+1) % cage_n] for i in range(cage_n)]
    star_edges = [[cage_n + i, cage_n + (i + 1) % star_n] for i in range(star_n)]
    
    edges = np.array(cage_edges + star_edges)

    plot = mp.plot(points, shading={"point_size": 0.1})
    plot.add_edges(points, edges)

display(cage_n, cage_points, star_n, star_points)

# Compute Harmonic Coordinates

In [ ]:
# compute harmonic weights for a given cage vertex
# returns a (# vertices, 1) column vector
def compute_hi(V, F, cage_indices, ci):
    
    L = igl.cotmatrix(V, F)
    b = np.zeros(V.shape[0])
    
    L = L.tolil()
    
    for i in cage_indices:
        L[i, :] = 0
        L[i, i] = 1
        b[i] = 1 if i == ci else 0
    
    L = L.tocsc()
    hi = sp.linalg.spsolve(L, b)

    return hi

In [ ]:
# a different version - using igl's harmonic function
def compute_hi2(V, F, cage_indices, ci):
    b = np.array(cage_indices)

    index = 0
    for i in range(len(cage_indices)):
        if i == ci:
            index = i
    
    bc = np.zeros((len(cage_indices), 1))
    bc[index] = 1.0
    
    hi = igl.harmonic(V, F, b, bc, 1)

    return hi.flatten()

In [ ]:
def compute_H(V, F, cage_indices):
    H = np.zeros((V.shape[0], len(cage_indices)))

    for i, ci in enumerate(cage_indices):
        hi = compute_hi(V, F, cage_indices, ci)
        H[:, i] = hi

    return H

In [ ]:
H = compute_H(v, f, cage_indices)

row_sums = np.sum(H, axis=1)
print(f"Min sum: {row_sums.min()}, Max sum: {row_sums.max()}") # each row's sum should be (close to) 1.

# Deformation

In [ ]:
# Update handles (four corners of the square) to deform cage
# After moving the handle, interpolate other cage vertices as necessary
# e.g., if we update the upper-left corner, interpolate the points along the left/top sides of the square

def deform_cage(delta):
    """
    deform cage by updating the handles and interpolating rest of the cage vertices
    input is delta of shape (4 x 3), each row corresonding to the displacement for corner i
    """
    
    v_copy = v.copy()

    UL, LL, LR, UR = 0, 1, 2, 3 # upper left, lower left, ...

    corners = [v[UL] + delta[0], v[LL] + delta[1], v[LR] + delta[2], v[UR] + delta[3]]

    # left side: interpolate between upper-left and lower-left
    y_top = v[UL][1]
    y_bot = v[LL][1]
    
    for idx in sides["left"]:
        y_orig = v[idx][1]
        t = (y_orig - y_top) / (y_bot - y_top)
        new_pos = (1 - t) * corners[0] + t * corners[1]
        v_copy[idx][:2] = new_pos[:2]

    # right side: interpolate between lower-right to upper-right
    y_top = v[UR][1]
    y_bot = v[LR][1]
    
    for idx in sides["right"]:
        y_orig = v[idx][1]
        t = (y_orig - y_top) / (y_bot - y_top)
        new_pos = (1 - t) * corners[3] + t * corners[2]
        v_copy[idx][:2] = new_pos[:2]

    # top side: interpolate between upper-left to upper-right
    x_left = v[UL][0]
    x_right = v[UR][0]
    
    for idx in sides["top"]:
        x_orig = v[idx][0]
        t = (x_orig - x_left) / (x_right - x_left)
        new_pos = (1 - t) * corners[0] + t * corners[3]
        v_copy[idx][:2] = new_pos[:2]

    # bottom side: interpolate between lower-left to lower-right
    x_left = v[LL][0]
    x_right = v[LR][0]
    
    for idx in sides["bottom"]:
        x_orig = v[idx][0]
        t = (x_orig - x_left) / (x_right - x_left)
        new_pos = (1 - t) * corners[1] + t * corners[2]
        v_copy[idx][:2] = new_pos[:2]

    return v_copy

In [ ]:
delta = np.zeros((4, 3))

# deform cage by setting displacements for the handles
delta[0] = [-0.3, 0, 0] # upper left
delta[1] = [0.3, 0, 0] # lower left
delta[2] = [0.3, 0, 0] # lower right
delta[3] = [-0.3, 0, 0] # upper right

v_copy = deform_cage(delta)

In [ ]:
# display the deformed cage (the shape is not deformed yet)
display(cage_n, v_copy[cage_indices], star_n, star_points)

In [ ]:
def deform_shape(v, H):    
    cage_deformed = v[cage_indices]
    
    v_deformed = H @ cage_deformed
    star_deformed = v_deformed[star_indices]
    
    display(cage_n, cage_deformed, star_n, star_deformed)

In [ ]:
deform_shape(v_copy, H) # display the deformed shape